# Classificador de Quebra de Parágrafo

Como os processos judiciais são em sua maioria originados de digitalização, é muito comum não saber se a terminação de uma linha diz respeito a um novo parágrafo ou não. Isso dificulta a segmentaçao dos conteúdos de peças em parágrafos e, em razão disso, torna-se complicado segmentar frases coesas e conteúdos relacionados. O conteúdo copiado a seguir, foi editado, para que as quebras de parágrafo ficassem destacadas com o símbolo de parágrafo.

```
Eis, na íntegra, o r. despacho de id. 29e8a8a, mediante o qual foi denegado\n 


seguimento ao recurso interposto pela ora agravante: §\n


CONSIDERAÇÕES PRELIMINARES §\n


O recorrente indica a existência de divergência jurisprudencial \n


entre as turmas deste Regional relativa aos temas Plano de Incentivo ao Desligamento \n


Voluntário - quitação das verbas rescisórias e natureza salarial do auxílio alimentação. \n


No entanto, em relação ao primeiro tema, não vislumbro a \n

```

Mascarado


```
Eis, na [MASK], o r. despacho de id. 29e8a8a, mediante o qual foi denegado\n 


seguimento ao recurso [MASK] pela ora agravante: §\n


CONSIDERAÇÕES PRELIMINARES §\n


O recorrente indica a existência de [MASK] jurisprudencial \n


entre as turmas deste [MASK] relativa aos temas Plano de Incentivo ao Desligamento \n


[MASK] - quitação das [MASK] rescisórias e natureza [MASK] do auxílio alimentação. \n


No entanto, em relação ao [MASK] tema, não vislumbro a \n

```

Perceba que, apesar de haver quebra de linha (`\n`), elas não necessariamente indicam a quebra de parágrafos (`§`). 

A tarefa alvo este trabalho é reconhecer quando uma quebra de linha refere-se a uma quebra de parágrafo. Para isso, considera-se uma `n` tokens antes e após a quebra de linha. As quebras de linha concatenadas serão substituídas por apenas uma quebra de linha. Os `n` tokens anteriores e posteriores serão coletados no fluxo do texto até que se encontre uma quebra de linha. Por exemplo, no texto anterior se considerarmos a terceira quebra de linha como alvo da classificação, teríramos o seguinte texto como entrada:

```
CONSIDERAÇÕES PRELIMINARES §\n


O recorrente indica a existência de divergência jurisprudencial \n
```

## Levantamento de Amostras

### Carrega o json com amostra rotuladas

In [2]:
import os
import json
import pandas as pd
import tabulate
from IPython.display import HTML, display
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, Button, HBox, VBox, Textarea, Layout, Box

In [2]:
!cat ../data/raw/julgados_GMALR.parquet.dvc

outs:
- md5: b6f2e4cae27f534022dbe6bd32fb5f5d
  size: 3786850746
  isexec: true
  path: julgados_GMALR.parquet


In [28]:
def get_file_path(md5):
    return f'{md5[:2]}/{md5[2:]}'

In [29]:
file_path = get_file_path('b6f2e4cae27f534022dbe6bd32fb5f5d')

In [30]:
path = '/mnt/nas/tst_dados/'

In [31]:
df = pd.read_parquet(path+file_path)

In [32]:
df.head(3)

,NUMERO_PROCESSO,ANO,DTA_ENTRADA,PARTES,PARTES_AUTORAS,PARTES_RES,ADVOGADOS,ASSUNTOS,FUNDAMENTACOES,ADMISSIBILIDADE,RECURSO_REVISTA,AIRR,ACORDAO,CODIGO_DECISAO,DESCRICAO_DECISAO,DECISAO,DESPACHO,CPF_PARTES,CNPJ_PARTES
0,0000001-04.2018.5.04.0103,"2020,0",2020/04/07 03:34:33.000000000,ANDERSON MADRUGA DE QUADROS\nBANCO BRADESCO S.A.,BANCO BRADESCO S.A.,ANDERSON MADRUGA DE QUADROS,André Rodigheri\nMozart Victor Russomano Neto,Correção Monetária,Análise prejudicada,\n\n\t\n\t\t\t\t\n\t\t\t\t\t\t[image: brasão]\...,\n \n\n \n\n\nEXCELENTÍSSIMO(A) SENHOR(A) DESE...,\n \n \n\n\n \n1 \n\n\n \n\n\...,[EMPTY],895; 1009,Despacho disponibilizado no DEJT.; Negado prov...,Agravante:BANCO BRADESCO S.A. E OUTRO\nAdvogad...,None,"-3,-3,91296897087,91296897087","60746948000112,60746948000112,-3,-3"
1,0000001-16.2019.5.05.0014,"2022,0",2022/03/22 12:01:37.000000000,BANCO SANTANDER ( BRASIL ) S. A.\nROSILDA DOS ...,BANCO SANTANDER ( BRASIL ) S. A.,ROSILDA DOS ANJOS SANTANA TEIXEIRA,João Ribeiro Porto\nJosé Eymard Loguercio\nLua...,Multa Cominatória / Astreintes\nNegativa de Pr...,Problemas em arestos\nReexame de fatos e provas,\n\n\t\n\t\t\t\t\n\t\t\t\t\t\t[image: brasão]\...,\n \n\n \n\n\nEXCELENTÍSSIMO SENHOR DOUTOR DES...,\n \n\n \n\n\nEXCELENTÍSSIMO(A) SENHOR(A) DOUT...,[EMPTY],1054,"Negado seguimento ao recurso, por ausência de ...",Agravante:BANCO SANTANDER (BRASIL) S.A.\nAdvog...,None,"-3,-3,-3,-3,78455669500,78455669500","90400888000142,90400888000142,90400888000142,9..."
2,0000001-37.2021.5.20.0011,"2022,0",2022/04/28 07:30:38.000000000,JOSE SEVERO DE SOUZA\nMOSAIC FERTILIZANTES P&K...,VALE S.A.,JOSE SEVERO DE SOUZA\nMOSAIC FERTILIZANTES P&K...,Carlos Eduardo Reis Cleto\nNilton Correia\nLil...,Sem Valor,Sem Valor,\nPODER JUDICIÁRIO \nJUSTIÇA DO TRABALHO \nTRI...,\n \n\n1 \nSERGIPE PERNAMBUCO ALAGOAS ...,\n \n\nSERGIPE PERNAMBUCO ALAGOAS ...,[EMPTY],1054,"Negado seguimento ao recurso, por ausência de ...",[EMPTY],None,"-3,-3,65964772404","33592510000154,33931486000130,-3"


In [33]:
def transform_TST_df(df):
    data = []
    for row in df.itertuples():
        if row.ADMISSIBILIDADE != '[EMPTY]':
            data.append([row.NUMERO_PROCESSO, row.ADMISSIBILIDADE, 'ADMISSIBILIDADE', 'julgados_GMALR', 'TST']) 
        if row.RECURSO_REVISTA != '[EMPTY]':
            data.append([row.NUMERO_PROCESSO,row.RECURSO_REVISTA, 'RR', 'julgados_GMALR', 'TST']) 
        if row.AIRR != '[EMPTY]':
            data.append([row.NUMERO_PROCESSO,row.AIRR, 'AIRR', 'julgados_GMALR', 'TST']) 
        if row.ACORDAO != '[EMPTY]':
            data.append([row.NUMERO_PROCESSO,row.ACORDAO, 'ACORDAO', 'julgados_GMALR', 'TST']) 
        if row.DECISAO != '[EMPTY]':   
            data.append([row.NUMERO_PROCESSO,row.DECISAO, 'DECISAO', 'julgados_GMALR', 'TST'])
    return data

In [34]:
df_interm = pd.DataFrame(transform_TST_df(df), columns=['NUMERO_PROCESSO', 'TXT', 'COLUMN', 'ARQUIVO_ORIGEM','CORTE']); df_interm.head()

,NUMERO_PROCESSO,TXT,COLUMN,ARQUIVO_ORIGEM,CORTE
0,0000001-04.2018.5.04.0103,\n\n\t\n\t\t\t\t\n\t\t\t\t\t\t[image: brasão]\...,ADMISSIBILIDADE,julgados_GMALR,TST
1,0000001-04.2018.5.04.0103,\n \n\n \n\n\nEXCELENTÍSSIMO(A) SENHOR(A) DESE...,RR,julgados_GMALR,TST
2,0000001-04.2018.5.04.0103,\n \n \n\n\n \n1 \n\n\n \n\n\...,AIRR,julgados_GMALR,TST
3,0000001-04.2018.5.04.0103,Agravante:BANCO BRADESCO S.A. E OUTRO\nAdvogad...,DECISAO,julgados_GMALR,TST
4,0000001-16.2019.5.05.0014,\n\n\t\n\t\t\t\t\n\t\t\t\t\t\t[image: brasão]\...,ADMISSIBILIDADE,julgados_GMALR,TST


In [37]:
#df = pd.read_parquet('/mnt/home/fabraz/code/CNJ/prec_qualif/data/interim/data_v2_indicacao_prec_regex.parquet'); df
!cat /mnt/home/fabraz/code/CNJ/prec_qualif/data/interim/data_v2.parquet.dvc

outs:
- md5: c94dee74fe005d10656a48787ad1a0c3
  size: 530362001
  path: data_v2.parquet


In [230]:
path = '/mnt/nas/cnj-pnud_dados/'

In [231]:
file_path = get_file_path('c94dee74fe005d10656a48787ad1a0c3')

In [232]:
df = pd.read_parquet(path+file_path)

In [233]:
df.origem_sigla.value_counts()

Tribunal de Justiça do Estado de Rondônia       27361
Tribunal Regional Federal da Terceira Região      202
Name: origem_sigla, dtype: int64

In [234]:
def transform_CNJ_df(df):
    data = []
    for row in df.itertuples():
        corte = None
        if row.origem_sigla == 'Tribunal de Justiça do Estado de Rondônia':
            corte = 'TJRR'
        else:
            corte = 'TRF3'
#        if row.ADMISSIBILIDADE != '[EMPTY]':
        if len(row.inicial_process_text) > 20:
            data.append([row.numero, row.inicial_process_text, 'inicial_process_text', 'data_v2.parquet', corte]) 
        if len(row.sentenca_text) > 20:
            data.append([row.numero, row.sentenca_text, 'sentenca_text', 'data_v2.parquet', corte]) 
        if row.contestacao_text is not None and len(row.sentenca_text) > 20:
            data.append([row.numero, row.contestacao_text, 'contestacao_text', 'data_v2.parquet', corte]) 
    return data

In [235]:
df_interm2 = pd.DataFrame(transform_CNJ_df(df), columns=['NUMERO_PROCESSO', 'TXT', 'COLUMN', 'ARQUIVO_ORIGEM','CORTE']); df_interm2.head()

,NUMERO_PROCESSO,TXT,COLUMN,ARQUIVO_ORIGEM,CORTE
0,7007776-24.2020.8.22.0007,Ss\n\nDV Ac\n\nJEAN DE JESUS SILVA\n\nEXCELENT...,inicial_process_text,data_v2.parquet,TJRR
1,7007776-24.2020.8.22.0007,\nPODER JUDICIÁRIO DO ESTADO DE RONDÔNIA\n \n ...,sentenca_text,data_v2.parquet,TJRR
2,7010905-52.2020.8.22.0002,\nMUNICÍPIO DE ARIQUEMES\nPREF. MUNICIPAL DE A...,inicial_process_text,data_v2.parquet,TJRR
3,7010905-52.2020.8.22.0002,\nPODER JUDICIÁRIO DO ESTADO DE RONDÔNIA \n Tr...,sentenca_text,data_v2.parquet,TJRR
4,7010790-31.2020.8.22.0002,\nMUNICÍPIO DE ARIQUEMES\nPREF. MUNICIPAL DE A...,inicial_process_text,data_v2.parquet,TJRR


In [236]:
df = pd.concat([df_interm, df_interm2]); df

,NUMERO_PROCESSO,TXT,COLUMN,ARQUIVO_ORIGEM,CORTE
0,0000001-04.2018.5.04.0103,\n\n\t\n\t\t\t\t\n\t\t\t\t\t\t[image: brasão]\...,ADMISSIBILIDADE,julgados_GMALR,TST
1,0000001-04.2018.5.04.0103,\n \n\n \n\n\nEXCELENTÍSSIMO(A) SENHOR(A) DESE...,RR,julgados_GMALR,TST
2,0000001-04.2018.5.04.0103,\n \n \n\n\n \n1 \n\n\n \n\n\...,AIRR,julgados_GMALR,TST
3,0000001-04.2018.5.04.0103,Agravante:BANCO BRADESCO S.A. E OUTRO\nAdvogad...,DECISAO,julgados_GMALR,TST
4,0000001-16.2019.5.05.0014,\n\n\t\n\t\t\t\t\n\t\t\t\t\t\t[image: brasão]\...,ADMISSIBILIDADE,julgados_GMALR,TST
...,...,...,...,...,...
63941,5002563-97.2019.4.03.6141,ADVOCACIA PREVIDENCIÁRIA \n\nEXCELENTÍSSIMO (A...,inicial_process_text,data_v2.parquet,TRF3
63942,5002563-97.2019.4.03.6141,Poder Judiciário\nJUSTIÇA FEDERAL DE PRIMEIRO ...,sentenca_text,data_v2.parquet,TRF3
63943,5020166-91.2018.4.03.6183,Ro Xuiz Coreto\n\n— ADVOCACIA\n\n\n\nEXCELENTÍ...,inicial_process_text,data_v2.parquet,TRF3
63944,5020166-91.2018.4.03.6183,Poder Judiciário\nJUSTIÇA FEDERAL DE PRIMEIRO ...,sentenca_text,data_v2.parquet,TRF3


In [237]:
#df

In [238]:
def sample_equal(df, column, n=400, random_state=42):
    samples = []
    for corte in df[column].unique():
        samples.append(df[df[column]==corte].sample(n=n,random_state=random_state))
    return pd.concat(samples)

In [239]:
df = sample_equal(df, 'CORTE', n=400); df

,NUMERO_PROCESSO,TXT,COLUMN,ARQUIVO_ORIGEM,CORTE
88492,0100280-41.2018.5.01.0342,\n \n \n A C Ó R D Ã O\n \...,ACORDAO,julgados_GMALR,TST
35563,0001606-71.2017.5.09.0073,\nPoder Judiciário Justiça do Trabalho Tribuna...,DECISAO,julgados_GMALR,TST
54035,0010678-13.2017.5.15.0042,\n\n\n\n \n \n\n\n \n\n\n \nAv. Presidente Va...,RR,julgados_GMALR,TST
57821,0010887-77.2020.5.18.0014,\n\n\t\n\t\t\t\t\n\t\t\t\t\t\t[image: brasão]\...,ADMISSIBILIDADE,julgados_GMALR,TST
63028,0011229-54.2021.5.18.0014,\nPODER JUDICIÁRIO \nJUSTIÇA DO TRABALHO \nTRI...,ADMISSIBILIDADE,julgados_GMALR,TST
...,...,...,...,...,...
63810,5001223-32.2020.4.03.6126,Poder Judiciário\nJUSTIÇA FEDERAL DE PRIMEIRO ...,sentenca_text,data_v2.parquet,TRF3
63928,5015990-35.2019.4.03.6183,EXCELENTÍSSIMO(A) SENHOR(A) DOUTOR(A) JUIZ(A) ...,inicial_process_text,data_v2.parquet,TRF3
63823,5000414-33.2020.4.03.6129,Poder Judiciário\nJUSTIÇA FEDERAL DE PRIMEIRO ...,sentenca_text,data_v2.parquet,TRF3
63747,5011593-70.2019.4.03.6105,Poder Judiciário\nJUSTIÇA FEDERAL DE PRIMEIRO ...,sentenca_text,data_v2.parquet,TRF3


In [240]:
pd.DataFrame(df.CORTE.value_counts())

,CORTE
TJRR,400
TST,400
TRF3,400


In [241]:
import re
def remove_extra_break_lines(text):
    '''
    remove extra breaklines from text, leaving with a single one
    '''
    _str = re.sub(r"(\s+\n)+","\n",text)
    _str = re.sub(r"^\n+","",_str)
    return _str

In [242]:
inp = '\n \n\n \n\n\n 1 \n\n\nEXMº(ª) SR.(ª) DR.(ª) JUÍZ(A) DESEMBARGADOR(A) VICE PRESIDENTE(A) DO TRIBUNAL REGIONAL DO TRABALHO DA QUINTA REGIÃO. \n\n\n \n\n\n \n\n\n \n\n\n \n\n\n \n\n\n \n\n\n \n\n\n \n\n\n      CLEUNILDA JESUS SILVA, já \ni\n'

out = ' 1\nEXMº(ª) SR.(ª) DR.(ª) JUÍZ(A) DESEMBARGADOR(A) VICE PRESIDENTE(A) DO TRIBUNAL REGIONAL DO TRABALHO DA QUINTA REGIÃO.\n      CLEUNILDA JESUS SILVA, já\ni\n'

result = remove_extra_break_lines(inp)

assert result == out, f'Test failed! For input: \n{inp}\nthe output should be: \n{out}\nbut comes:\n{result}'

In [243]:
df['TXT'] = df['TXT'].apply(remove_extra_break_lines)

In [244]:
df.reset_index(inplace=True)

In [245]:
df

,index,NUMERO_PROCESSO,TXT,COLUMN,ARQUIVO_ORIGEM,CORTE
0,88492,0100280-41.2018.5.01.0342,A C Ó R D Ã O\n 4ª Turma\n ...,ACORDAO,julgados_GMALR,TST
1,35563,0001606-71.2017.5.09.0073,Poder Judiciário Justiça do Trabalho Tribunal ...,DECISAO,julgados_GMALR,TST
2,54035,0010678-13.2017.5.15.0042,"Av. Presidente Vargas, n° 2001 - Cj.156 - Ribe...",RR,julgados_GMALR,TST
3,57821,0010887-77.2020.5.18.0014,\t\t\t\t\t\t[image: brasão]\n\t\t\t\t\t\t\tPOD...,ADMISSIBILIDADE,julgados_GMALR,TST
4,63028,0011229-54.2021.5.18.0014,PODER JUDICIÁRIO\nJUSTIÇA DO TRABALHO\nTRIBUNA...,ADMISSIBILIDADE,julgados_GMALR,TST
...,...,...,...,...,...,...
1195,63810,5001223-32.2020.4.03.6126,Poder Judiciário\nJUSTIÇA FEDERAL DE PRIMEIRO ...,sentenca_text,data_v2.parquet,TRF3
1196,63928,5015990-35.2019.4.03.6183,EXCELENTÍSSIMO(A) SENHOR(A) DOUTOR(A) JUIZ(A) ...,inicial_process_text,data_v2.parquet,TRF3
1197,63823,5000414-33.2020.4.03.6129,Poder Judiciário\nJUSTIÇA FEDERAL DE PRIMEIRO ...,sentenca_text,data_v2.parquet,TRF3
1198,63747,5011593-70.2019.4.03.6105,Poder Judiciário\nJUSTIÇA FEDERAL DE PRIMEIRO ...,sentenca_text,data_v2.parquet,TRF3


In [246]:
df.to_parquet('../data/raw/df_origem_paragrafos.parquet')
#df = df.iloc[:,1:]

In [247]:
def get_samples_left(lines, spam, index):
    left = index - spam if index - spam >=0 else 0
    return ' '.join([item for item in lines[left:index]])

def get_samples_right(lines, spam, index):
    right = index + spam + 1 if index + spam + 1 < len(lines) else len(lines)-1
    return ' '.join([item for item in lines[index+1:right]])

samples = []
for row in df.itertuples():
    index = 1
    lines = row.TXT.splitlines(True)    
    spam = 3
    while index < len(lines)-1:
        samples.append([row.index, get_samples_left(lines, spam, index),lines[index], get_samples_right(lines, spam, index), index])
        index += 1

In [248]:
len(samples)

294088

In [249]:
from random import seed, sample
seed(42)

In [262]:
df_paragrafos = pd.DataFrame(sample(samples, 50*1000), columns=['index_texto_original','anterior', 'alvo', 'posterior', 'linha'])

In [251]:
#df_amostras_paragrafos.to_parquet('../data/raw/df_paragrafos.parquet')

In [270]:
def get_source_text(index, df_paragrafos, df_origem_paragrafos):
    '''
    Return the raw text as lf split array used to create paragraph sample
    '''

    index_texto_original = df_amostras_paragrafos.loc[index][0]
    txt = df_origem_paragrafos[df_origem_paragrafos['index']==index_texto_original].iloc[0,2]
    return txt

In [273]:
get_source_text(185, df_paragrafos, df).split('\n')[10:30]

['de participar de audiências e apresentar defesa nos autos do processo nº 0024146-',
 '31.2014.8.22.0001, parte autora: SERGIO AURES BATISTA e réu: Banco do Brasil, excetuando-se',
 'o poder de substabelecer para outrem.',
 'São Paulo, 16 de março de 2015.',
 'TATO PISSINI',
 'SP 261.030',
 'Baruet » SP | Ale',
 'Rio Megros tál',
 '= Aiohiavllo » 64540 » FoneiFeme (IT) SB94 3800',
 '8180',
 'SIGNORPÉis. smcMARQUESINI',
 'g.; ui =;i u s',
 'SUBSTABELECIMENTO',
 '(COM RESERVA DE PODERES)',
 'Substabeleço, com reserva de iguais poderes, na pessoa de',
 'CAROLINA GIOSCIA LEAL DE MELO, brasileira, advogada, inscrita na OAB/RO 2.592, com escritório',
 'profissional à Rua Marechal Deodoro, 2712, Sala C, CEP: 76801260',
 '(69) 3229-2898/(59) 8444-7328, e-mail: carolinaleai.advocacia@gmaii.com, para o fim especifico',
 'de participar de audiências e apresentar defesa nos autos do processo n9 0024146-',
 '31.2014.8.22.0001, parte autora: SERGIO AURES BATISTA e réu: Banco do Brasil, excetuando-s

### Carrega as amostras ja rotuladas e mostra a quantidade

In [81]:
import os
import json
import pandas as pd
import tabulate
from IPython.display import HTML, display
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, Button, HBox, VBox, Textarea, Layout, Box

In [82]:
label_file_path = '../data/raw/labels.json'

In [83]:
try:
    with open(label_file_path) as json_file:
        data = json.load(json_file)
except:
    print("An exception occurred when trying to open %s" % label_file_path)
    #print(e)
    data = {}
print(len(data.keys()))

2667


### Carrega dados sem pre-processamento

In [85]:
df_paragrafos = pd.read_parquet('../data/raw/df_paragrafos.parquet') 
print(df_raw.shape)

(50000, 5)


In [5]:
# print(merge_columns(df_raw.iloc[44563,:]))

### Filtra amostras de peças sem as rotuladas

In [88]:
def df_to_table(table):
    display(HTML(tabulate.tabulate(table, tablefmt='html')))
    
def merge_columns(row):
    return row['anterior']+ '§'+row['alvo']+'§'+row['posterior']
    
def generate_sample(df_raw, data):
    #remove already labeled sample
#    print(df_raw.shape)
    #print(data)
    df_raw = df_raw.filter(items = set(df_raw.index) - set(data.keys()), axis=0)
#    df_raw = df_raw[~df_raw.index.isin(list(map(int, data.keys())))]
#    print(df_raw.shape)
    sample = df_raw.sample(n=1,replace=True)
    #display(sample)
#    print(sample.text.iloc[0][-600:])    
#    df_to_table(sample[['piece_id', 'text']].values.tolist())
    return sample.index[0], merge_columns(sample.iloc[0,:])
#    return df_raw, sample.piece_id.iloc[0]

In [264]:
generate_sample(df_paragrafos, data)

(49798,
 'teve ciência de que a transação abrangia o pedido de horas extras (diárias\n e em ocorrências) e seus reflexos e, assim mesmo, deu quitação plena e\n geral por esses objetos do pedido, o que resulta na extinção do feito, sem\n§resolução do mérito, quanto ao pedido de horas extras diárias e por\n§ocorrências fora do horário de expediente. Registre-se que não constam\n ressalvas no termo de conciliação (v. item 4, fl. 19-verso), onde deveriam\n constar as horas extras excedentes, mas de mesma origem daquelas\n')

## Rotulação

### Rotula Amotra

In [90]:
def label_sample(id, label, data):
    data[id] = label
    print('label_sample ',id, label, len(data))
    return data

In [91]:
thisandthat = ['QuebraPar_Arg', 'NaoQuebraPar_Arg','QuebraPar_NaoArg', 'NaoQuebraPar_NaoArg', 'Erro']

sample_id, text = generate_sample(df_raw, data)

switch = [Button(description=name) for name in thisandthat]

sample = widgets.IntText(
    value=sample_id,
    description='sample_id:',
    disabled=False
)

saveBtn = Button(description='Save')

dataTextArea = Textarea(value=str(data),
                            description='',
                            rows=1,
                            layout=Layout(width="auto"))


l = [items for items in switch]

l.append(saveBtn)
l.append(sample)

combined = HBox(l)

def save_clicked(btn):
    with open('../data/raw/labels.json', 'w') as outfile:
        json.dump(eval(dataTextArea.value), outfile)
        print('file save')

def upon_clicked(btn):
    label = 0
    if btn.description==thisandthat[0]:
        label = 0
    elif btn.description==thisandthat[1]:
        label = 1
    elif btn.description==thisandthat[2]:
        label = 2        
    elif btn.description==thisandthat[3]:
        label = 3                
    else:
        label = -1 
    
    data = label_sample(sample.value, label, eval(dataTextArea.value))
    sample_id, text = generate_sample(df_raw, data)
    comm_text.value = text
    comm_text.rows = 10
    sample.value = sample_id
    dataTextArea.value=str(data)

for n in range(len(thisandthat)):
    switch[n].on_click(upon_clicked)
    
saveBtn.on_click(save_clicked)

display(combined)

comm_text = Textarea(value=text,
                            placeholder='OK',
                            description='',
                            rows=10,                            
                            layout=Layout(width="auto"))

display(comm_text)

Textarea(value='HIPNet / DATAPREV - Empresa de Tecnologia e Informações da Previdência Social Páginafçde 1 “os…